In [1]:
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import utils
# load ascii text and covert to lowercase
filename = "AllD/StalkerL 3v1.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

# Сортируем по буквам
chars=list(['\n','\xa0','-',' ',',','.','а','б','в','г','д','е','ж','з','и','к','л','м','н','о','п','р','с','т','у','ф','х','ц','ч','ш','щ','ы','ь','э','ю','я'])

char_to_int = dict((c, i) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  1809304
Total Vocab:  36


In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1' #-1 не использовать gpu

In [2]:
!nvidia-smi

Wed Dec 18 11:48:54 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:1A:00.0 Off |                    0 |
| N/A   31C    P0    41W / 300W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:1C:00.0 Off |                    0 |
| N/A   28C    P0    42W / 300W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [4]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 256
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  1809048


In [5]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = utils.to_categorical(dataY)


In [6]:
from tensorflow.keras.optimizers import RMSprop

# define the LSTM model
model = models.Sequential()
model.add(layers.LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(layers.LSTM(256, return_sequences=True))
model.add(layers.Dropout(0.05))
model.add(layers.Dense(y.shape[1], activation='softmax'))
optimizer = RMSprop(lr=0.01)
# адам - для лохов(с) интернет
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
# define the checkpoint
filepath="StalkerOnly1LBiggerNeuron512-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


In [ ]:
# fit the model
model.fit(X, y, epochs=15, batch_size=4096, callbacks=callbacks_list)

### Генерируем текст

In [7]:
# load the network weights
filename = "Stalker3v1LBiggerNeuron-06-1.9413-bigger.hdf5"
model.load_weights(filename)
#model.compile(loss='categorical_crossentropy', optimizer='adam')
optimizer = RMSprop(lr=0.01)
# адам - для лохов(с) интернет
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [10]:
int_to_char = dict((i, c) for i, c in enumerate(chars))
import sys
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" вируя между артефактами, которые не успел или не смог, без дополнительнои защиты, взять - их было еще много, я набрал всего два десятка, больше не влезало в контеинеры.
     - видно или только слышно. - спросил я, приседая на самом краю и вглядываясь в жел "


KeyError: 7504